In [7]:
import math
# 假設 TWD97 座標為 (tx, ty)
# 假設 WGS84 座標為 (x, y)
# X97=X67+ 828，Y97=Y67- 207
169784.98804265473,2543493.999829864
tx = 169784.98804265473
ty = 2543493.999829864
y = ty * 0.00000899823754
x = 121 + (tx - 250000) * 0.000008983152841195214 / math.cos(math.radians(y))

In [8]:
print(x, y)

120.21783988882247 22.886963192033836


In [13]:
ty = y/0.00000899823754
tx = (x-121)*math.cos(math.radians(y))/0.000008983152841195214 + 250000


In [14]:
print(tx,ty)

169784.9880426546 2543493.999829864
